In [1]:
# coding: utf-8
import os
import xlrd
import re
import csv
import fasttext
import traceback
import random
import utils
import json
import jieba
import pandas as pd
from utils import fixText

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

In [2]:
with open('data/train.tsv', 'r') as f:
    content = f.readlines()
    random.shuffle(content)
    train = content[0:len(content) // 5 * 4]
    test = content[len(content) // 5 * 4:]
    
    with open('data/old_train_set.txt', 'w') as out:
        for c in train:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            out.write(c + ' ' + label + '\n')
    with open('data/old_test_set.txt', 'w') as out:
        for c in test:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            # c = ''.join(c.split(' '))
            out.write(c + ' ' + label + '\n')
        

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built successfully.


In [3]:
config = getConfig()
model = fasttext.train_supervised(
    input = './data/old_train_set.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

Read 4M words
Number of words:  73112
Number of labels: 43
Progress: 100.0% words/sec/thread:  800887 lr:  0.000000 avg.loss:  0.437031 ETA:   0h 0m 0s


In [11]:
total = 0
true_tag = 0
with open('./data/old_test_set.txt', 'r') as f:
    content = f.readlines()
    total = len(content)
    for c in content:
        res = re.search(r'__label__(.*)[^\r\n]', c)
        label = c[res.span()[0]:res.span()[1]]
        c = re.sub(r'__label__(.*)', '', c)
        predict = model.predict(c[:-1])[0][0]
        
        print(model.predict(c[:-1]))
        # print(predict, label)
        if(predict == label):
            true_tag += 1
        break
#print('tag level 1 accurate: {}% ({}/{})'.format(true_tag * 100 / total, true_tag, total))

(('__label__街道',), array([0.90377563]))
